In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import datetime as dt
import requests
import json

In [2]:
urls_lost = []
urls_found = []
urls_reunited = []

city = "oakland"
state = "ca"
page_no = 1


"""
city = input("Please type a city: ")
state = input("Please type the 2-letter state abbreviation: ")
"""

# status codes: 100 = lost; 101 = found; 102 = reunited

for status in range(100,103):
    url = 'https://www.pawboost.com/lost-found-pets/' + \
        city + '-' + state + '-/all-lost-found-stray-reunited-cats/page-' + str(page_no) + \
        '?LfdbFeedStatusForm%5Bstatus%5D%5B%5D=' + str(status) + '&LfdbFeedStatusForm%5Baddress%5D=' + \
        city + '%2C%20' + state + \
        '&LfdbFeedStatusForm%5Bradius%5D=15&LfdbFeedStatusForm%5Bspecies%5D=Cat' + \
        '&LfdbFeedStatusForm%5BsortAttribute%5D=distance&LfdbFeedStatusForm%5BdateRange%5D=90'


    response = requests.get(url)
    html = response.content

    soup = BeautifulSoup(html, "html.parser")
    soup.prettify()

    if status == 100:
        print("Lost cats:")
    elif status == 101:
        print("Found cats:")
    elif status == 102:
        print("Reunited w/ family:")
    else:
        print("error")
    
    print(url)
    print("URLs found:")
    
    for anchor in soup.findAll('a', attrs={'class': 'btn btn-primary width-full'}, href=True):
        print(anchor['href'])
        if status == 100:
            urls_lost.append(anchor['href'])
        elif status == 101:
            urls_found.append(anchor['href'])
        elif status == 102:
            urls_reunited.append(anchor['href'])
    

Lost cats:
https://www.pawboost.com/lost-found-pets/oakland-ca-/all-lost-found-stray-reunited-cats/page-1?LfdbFeedStatusForm%5Bstatus%5D%5B%5D=100&LfdbFeedStatusForm%5Baddress%5D=oakland%2C%20ca&LfdbFeedStatusForm%5Bradius%5D=15&LfdbFeedStatusForm%5Bspecies%5D=Cat&LfdbFeedStatusForm%5BsortAttribute%5D=distance&LfdbFeedStatusForm%5BdateRange%5D=90
URLs found:
https://www.pawboost.com/landing/pet/22WBSUN0blBIxmHGFrrgo9gyEyqkP-Gc/lost-delilah-murrieta-ca-92563
https://www.pawboost.com/landing/pet/qpko2mdlJohEUvb_XQxixOACMHcpTnp-/lost-juliette-oakland-ca-94610
https://www.pawboost.com/landing/pet/Ez21xfLD1CK9tQ_tR5G_C7IABkt4A-3b/lost-tom-oakland-ca-94606
https://www.pawboost.com/landing/pet/5ZanwqHyXCq_UJ0auwhsGhrEMuEMqKZ2/lost-bluashley-oakland-ca-94609
https://www.pawboost.com/landing/pet/yN7BzhbxsJ2LgUj2b_9B83aUGuFOLIsx/lost-macaroni-oakland-ca-94607
https://www.pawboost.com/landing/pet/oP9EWgMv0bkMT_JokNLRZfN_bLK-IXco/lost-mickey-alameda-ca-94501
https://www.pawboost.com/landing/pet/5K

In [3]:
for status in range(100,103):
    if status == 100:
        length = len(urls_lost)
        active_list = urls_lost
    elif status == 101:
        length = len(urls_found)
        active_list = urls_found
    elif status == 102:
        length = len(urls_reunited)
        active_list = urls_reunited
    
    pet_url = []
    pet_id = []
    pet_name = []
    pet_gender = []
    pet_status = []
    pet_description = []
    pet_address = []
    pet_address_clean = []
    
    # first url is irrelevant due to featured pet
    for i in range(1,length):
        new_url = active_list[i]
        pet_url.append(new_url)
        
        response = requests.get(new_url)
        html = response.content

        soup = BeautifulSoup(html, "html.parser")
        soup.prettify()
        # print(response)

        item_no = []
        h2_tags = []
        p_tags = []

        for i in range(0, 12):
            paragraphs = soup.find_all("h2")[i].text
            item_no.append(i)
            h2_tags.append(paragraphs)
    
        for i in range(0, 20):
            paragraphs = soup.find_all("p")[i].text
            item_no.append(i)
            p_tags.append(paragraphs)

        # for viewing which text fields are relevant
        results_h2 = pd.DataFrame({"text": h2_tags})
        results_p = pd.DataFrame({"text": p_tags})

        # pet ID
        pet_id.append(results_h2['text'][3]) 
        
        # pet name
        if status == 100:
            pet_name.append(results_h2['text'][4])
        elif status == 101:
            pet_name.append("Unknown")
        elif status == 102:
            pet_name.append(results_h2['text'][4])
        
        # gender
        if status == 100:
            pet_gender.append(results_h2['text'][5])
        elif status == 101:
            pet_gender.append(results_h2['text'][4])
        elif status == 102:
            pet_gender.append(results_h2['text'][5])
        
        # status
        if status == 100:
            pet_status.append(results_p['text'][4])
        elif status == 101:
            pet_status.append(results_p['text'][3])
        elif status == 102:
            pet_status.append(results_p['text'][4])
            
        # description
        if status == 100:
            pet_description.append(results_p['text'][10])
        elif status == 101:
            pet_description.append(results_p['text'][9])
        elif status == 102:
            pet_description.append(results_p['text'][8])
        
        # address, if available
        if status == 100:
            address = results_h2['text'][8] + " " + results_h2['text'][7]
        elif status == 101:
            address = results_h2['text'][7] + " " + results_h2['text'][6]
        elif status == 102:
            address = results_h2['text'][7]
               
        pet_address.append(address)
        
        address_clean = address.replace(' ', '+').replace('&', 'and')
        pet_address_clean.append(address_clean)
        # print(address_clean)
        
        if status == 100:
            details_lost = pd.DataFrame({"url": pet_url,
                "ref_no": pet_id,
                "pet_name": pet_name,
                "gender": pet_gender,
                "status": pet_status,
                "description": pet_description,
                "address": pet_address,
                "address_clean": pet_address_clean})
        elif status == 101:
            details_found = pd.DataFrame({"url": pet_url,
                "ref_no": pet_id,
                "pet_name": pet_name,
                "gender": pet_gender,
                "status": pet_status,
                "description": pet_description,
                "address": pet_address,
                "address_clean": pet_address_clean})
        elif status == 102:
            details_reunited = pd.DataFrame({"url": pet_url,
                "ref_no": pet_id,
                "pet_name": pet_name,
                "gender": pet_gender,
                "status": pet_status,
                "description": pet_description,
                "address": pet_address,
                "address_clean": pet_address_clean})
    

In [4]:
details_lost

,url,ref_no,pet_name,gender,status,description,address,address_clean
0,https://www.pawboost.com/landing/pet/qpko2mdlJ...,5493951,Juliette,Female,LOST,Petite black female cat. Very affectionate. At...,"Near Burk St & Euclid Ave Oakland, CA 94610","Near+Burk+St+and+Euclid+Ave+Oakland,+CA+94610"
1,https://www.pawboost.com/landing/pet/Ez21xfLD1...,5451799,Tom,Male,LOST,Grey domestic cat with grey eyes. Super friend...,"Near E 20th St & 5th Ave Oakland, CA 94606","Near+E+20th+St+and+5th+Ave+Oakland,+CA+94606"
2,https://www.pawboost.com/landing/pet/5ZanwqHyX...,5492462,Blu Ashley,Female,LOST,"Fluffy grey maincoon. Green eyes, Blue/green ...","Hoover-Foster Oakland, CA 94609","Hoover-Foster+Oakland,+CA+94609"
3,https://www.pawboost.com/landing/pet/yN7Bzhbxs...,5461933,Macaroni,Male,LOST,Large tall orange tabby,"Prescott Oakland, CA 94607","Prescott+Oakland,+CA+94607"
4,https://www.pawboost.com/landing/pet/oP9EWgMv0...,5461739,Mickey,Female,LOST,Grey/Blue short hair,"Around Ralphs Alameda, CA 94501","Around+Ralphs+Alameda,+CA+94501"
5,https://www.pawboost.com/landing/pet/5K0IWzh00...,5483685,Robin,Female,LOST,Taken from front porch !\nlong haired black cat,"Middle Alameda Alameda, CA 94501","Middle+Alameda+Alameda,+CA+94501"
6,https://www.pawboost.com/landing/pet/iaW_M_2P-...,5496170,Tanya,Female,LOST,"Black cat, she is about ten pounds but thick l...","Middle Alameda Alameda, CA 94501","Middle+Alameda+Alameda,+CA+94501"
7,https://www.pawboost.com/landing/pet/EOdBmNvCl...,5467708,Pompey,Male,LOST,"Pure black, short hair, green eyes, approximat...","Near E 27th St & 19th Ave Oakland, CA 94606","Near+E+27th+St+and+19th+Ave+Oakland,+CA+94606"
8,https://www.pawboost.com/landing/pet/OPvZQNBUk...,5445610,Chloe,Female,LOST,NEIGHBORHOOD LAST SEEN,"Central Alameda Alameda, CA 94501","Central+Alameda+Alameda,+CA+94501"
9,https://www.pawboost.com/landing/pet/vGvUk8IAy...,5431882,Toru,Male,LOST,A gray tabby with yellow green eyes. He is ver...,"Oak Tree Oakland, CA 94601","Oak+Tree+Oakland,+CA+94601"


In [5]:
details_found

,url,ref_no,pet_name,gender,status,description,address,address_clean
0,https://www.pawboost.com/landing/pet/17CqQ9hRj...,5499798,Unknown,Female,FOUND,I think it might be a calico cat... Not to sur...,"Near la Salle Ave & King Ave Piedmont, CA 94610","Near+la+Salle+Ave+and+King+Ave+Piedmont,+CA+94610"
1,https://www.pawboost.com/landing/pet/-kF0OoECq...,5406229,Unknown,Male,FOUND,Very friendly and sweet.,"Near 47th Ave & Ygnacio Ave Oakland, CA 94601","Near+47th+Ave+and+Ygnacio+Ave+Oakland,+CA+94601"
2,https://www.pawboost.com/landing/pet/FOLymmKQb...,5445880,Unknown,Unknown,FOUND,Multi colored cat,"Near 67th Ave & Avenal Ave Oakland, CA 94605","Near+67th+Ave+and+Avenal+Ave+Oakland,+CA+94605"
3,https://www.pawboost.com/landing/pet/T1W061MY0...,5412303,Unknown,Female,FOUND,Orange white has collar with pink bow. Super f...,"Near Parker Ave & Outlook Ave Oakland, CA 94605","Near+Parker+Ave+and+Outlook+Ave+Oakland,+CA+94605"
4,https://www.pawboost.com/landing/pet/qXEhVVZT3...,5412800,Unknown,Female,FOUND,Super friendly and obviously lost from a famil...,"Near Parker Ave & Outlook Ave Oakland, CA 94605","Near+Parker+Ave+and+Outlook+Ave+Oakland,+CA+94605"
5,https://www.pawboost.com/landing/pet/M-8bw1Bz6...,5370159,Unknown,Female,FOUND,Calico Cat that I have been feeding every nigh...,"Near Coach Dr & Phaeton Dr Oakland, CA 94605","Near+Coach+Dr+and+Phaeton+Dr+Oakland,+CA+94605"
6,https://www.pawboost.com/landing/pet/QQXhIwWuq...,5447641,Unknown,Unknown,FOUND,Black cat with white chest and white paws.,"Potrero Hill San Francisco, CA 94107","Potrero+Hill+San+Francisco,+CA+94107"
7,https://www.pawboost.com/landing/pet/ZsDLbMKFt...,5470380,Unknown,Unknown,FOUND,NEIGHBORHOOD LAST SEEN,"Harlan-Estabrook-Castro San Leandro, CA 94577","Harlan-Estabrook-Castro+San+Leandro,+CA+94577"
8,https://www.pawboost.com/landing/pet/nXeOeZaF9...,5509632,Unknown,Male,FOUND,Sweet male cat who loves to purr and cuddle ju...,"Alamo Square San Francisco, CA 94117","Alamo+Square+San+Francisco,+CA+94117"
9,https://www.pawboost.com/landing/pet/mLVcy1OwJ...,5516397,Unknown,Male,FOUND,Orange tabby (assume it's a male). Very old/si...,"Alamo Square San Francisco, CA 94115","Alamo+Square+San+Francisco,+CA+94115"


In [6]:
details_reunited

,url,ref_no,pet_name,gender,status,description,address,address_clean
0,https://www.pawboost.com/landing/pet/oq00DJiba...,5485310,Jax,Male,SAFE,"Jax is a tabby colored cat with a white belly,...","Oakland, CA 94610","Oakland,+CA+94610"
1,https://www.pawboost.com/landing/pet/8voHTy_iZ...,5450995,Chad,Male,SAFE,Black and White cat (sometimes called a tuxedo...,"Oakland, CA 94606","Oakland,+CA+94606"
2,https://www.pawboost.com/landing/pet/uEmdVyeiT...,5429986,Justice,Male,SAFE,NEIGHBORHOOD LAST SEEN,"Oakland, CA 94607","Oakland,+CA+94607"
3,https://www.pawboost.com/landing/pet/MHr71Jv79...,5507282,Loki,Male,SAFE,Loki has one eye but is fully blind. His tail ...,"Oakland, CA 94610","Oakland,+CA+94610"
4,https://www.pawboost.com/landing/pet/mi8ie5x3y...,5396921,Luna,Female,SAFE,Microchipped. Spayed. Wearing a shiny purple c...,"Oakland, CA 94611","Oakland,+CA+94611"
5,https://www.pawboost.com/landing/pet/aFvQ2ZwZ-...,5467952,Romelu,Male,SAFE,"8 month old brown tabby kitten, no collar, but...","Oakland, CA 94608","Oakland,+CA+94608"
6,https://www.pawboost.com/landing/pet/bWsWhpu63...,5454579,Liba,Female,SAFE,Tortiseshell short hair female with light eyeb...,"Oakland, CA 94602","Oakland,+CA+94602"
7,https://www.pawboost.com/landing/pet/Uo6Jdtp_I...,5392063,Calico Bunny,Female,SAFE,"Calico kitty, 4 years old, weighs about 5lbs. ...","Oakland, CA 94601","Oakland,+CA+94601"
8,https://www.pawboost.com/landing/pet/JkKj-VEdK...,5450424,Raja,Female,SAFE,She’s a little sweet calico kitty with a cute ...,"Alameda, CA 94501","Alameda,+CA+94501"
9,https://www.pawboost.com/landing/pet/2HDWoUbGw...,5380186,Emi,Female,SAFE,10 year old blue cream point Siamese mix. Very...,"Berkeley, CA 94705","Berkeley,+CA+94705"


In [7]:
# need to do api call because coordinates could not be scraped from page
for status in range(100,103):
    if status == 100:
        length = len(urls_lost)
    elif status == 101:
        length = len(urls_found)
    elif status == 102:
        length = len(urls_reunited)
        
    latitude = []
    longitude = []
    
    for i in range(0,length-1):
        if status == 100:
            query_address = details_lost['address_clean'][i]
        elif status == 101:
            query_address = details_found['address_clean'][i]
        elif status == 102:
            query_address = details_reunited['address_clean'][i]
        
        parameters = query_address
        api_key = #REDACTED

        query_url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + \
            parameters + "&key=" + api_key

        print(f"finding coordinates for: {query_address}")
        #print(query_url)
        response = requests.get(query_url)
        gmap_json = response.json()

        try:
            print(gmap_json['results'][0]['geometry']['location']['lat'])
            latitude.append(float(gmap_json['results'][0]['geometry']['location']['lat']))
        except (KeyError, IndexError):
            print("could not find latitude")
            latitude.append(None)      
        try:
            print(gmap_json['results'][0]['geometry']['location']['lng'])
            longitude.append(float(gmap_json['results'][0]['geometry']['location']['lng']))
        except (KeyError, IndexError):
            print("could not find longitude")
            longitude.append(None)
            
        print("--------------------------")
        
    if status == 100:
        details_lost['latitude'] = latitude
        details_lost['longitude'] = longitude
    elif status == 101:
        details_found['latitude'] = latitude
        details_found['longitude'] = longitude
    elif status == 102:
        details_reunited['latitude'] = latitude
        details_reunited['longitude'] = longitude

print("Coordinate lookup completed")               

finding coordinates for: Near+Burk+St+and+Euclid+Ave+Oakland,+CA+94610
37.8091297
-122.2516412
--------------------------
finding coordinates for: Near+E+20th+St+and+5th+Ave+Oakland,+CA+94606
37.7997012
-122.2482853
--------------------------
finding coordinates for: Hoover-Foster+Oakland,+CA+94609
37.8207406
-122.272478
--------------------------
finding coordinates for: Prescott+Oakland,+CA+94607
37.812174
-122.2965778
--------------------------
finding coordinates for: Around+Ralphs+Alameda,+CA+94501
37.7753559
-122.2724372
--------------------------
finding coordinates for: Middle+Alameda+Alameda,+CA+94501
37.7526273
-122.2311322
--------------------------
finding coordinates for: Middle+Alameda+Alameda,+CA+94501
37.7526273
-122.2311322
--------------------------
finding coordinates for: Near+E+27th+St+and+19th+Ave+Oakland,+CA+94606
37.7959854
-122.231919
--------------------------
finding coordinates for: Central+Alameda+Alameda,+CA+94501
37.7703612
-122.2569641
------------------

In [8]:
details_lost

,url,ref_no,pet_name,gender,status,description,address,address_clean,latitude,longitude
0,https://www.pawboost.com/landing/pet/qpko2mdlJ...,5493951,Juliette,Female,LOST,Petite black female cat. Very affectionate. At...,"Near Burk St & Euclid Ave Oakland, CA 94610","Near+Burk+St+and+Euclid+Ave+Oakland,+CA+94610",37.809130,-122.251641
1,https://www.pawboost.com/landing/pet/Ez21xfLD1...,5451799,Tom,Male,LOST,Grey domestic cat with grey eyes. Super friend...,"Near E 20th St & 5th Ave Oakland, CA 94606","Near+E+20th+St+and+5th+Ave+Oakland,+CA+94606",37.799701,-122.248285
2,https://www.pawboost.com/landing/pet/5ZanwqHyX...,5492462,Blu Ashley,Female,LOST,"Fluffy grey maincoon. Green eyes, Blue/green ...","Hoover-Foster Oakland, CA 94609","Hoover-Foster+Oakland,+CA+94609",37.820741,-122.272478
3,https://www.pawboost.com/landing/pet/yN7Bzhbxs...,5461933,Macaroni,Male,LOST,Large tall orange tabby,"Prescott Oakland, CA 94607","Prescott+Oakland,+CA+94607",37.812174,-122.296578
4,https://www.pawboost.com/landing/pet/oP9EWgMv0...,5461739,Mickey,Female,LOST,Grey/Blue short hair,"Around Ralphs Alameda, CA 94501","Around+Ralphs+Alameda,+CA+94501",37.775356,-122.272437
5,https://www.pawboost.com/landing/pet/5K0IWzh00...,5483685,Robin,Female,LOST,Taken from front porch !\nlong haired black cat,"Middle Alameda Alameda, CA 94501","Middle+Alameda+Alameda,+CA+94501",37.752627,-122.231132
6,https://www.pawboost.com/landing/pet/iaW_M_2P-...,5496170,Tanya,Female,LOST,"Black cat, she is about ten pounds but thick l...","Middle Alameda Alameda, CA 94501","Middle+Alameda+Alameda,+CA+94501",37.752627,-122.231132
7,https://www.pawboost.com/landing/pet/EOdBmNvCl...,5467708,Pompey,Male,LOST,"Pure black, short hair, green eyes, approximat...","Near E 27th St & 19th Ave Oakland, CA 94606","Near+E+27th+St+and+19th+Ave+Oakland,+CA+94606",37.795985,-122.231919
8,https://www.pawboost.com/landing/pet/OPvZQNBUk...,5445610,Chloe,Female,LOST,NEIGHBORHOOD LAST SEEN,"Central Alameda Alameda, CA 94501","Central+Alameda+Alameda,+CA+94501",37.770361,-122.256964
9,https://www.pawboost.com/landing/pet/vGvUk8IAy...,5431882,Toru,Male,LOST,A gray tabby with yellow green eyes. He is ver...,"Oak Tree Oakland, CA 94601","Oak+Tree+Oakland,+CA+94601",37.782099,-122.229226


In [9]:
details_found

,url,ref_no,pet_name,gender,status,description,address,address_clean,latitude,longitude
0,https://www.pawboost.com/landing/pet/17CqQ9hRj...,5499798,Unknown,Female,FOUND,I think it might be a calico cat... Not to sur...,"Near la Salle Ave & King Ave Piedmont, CA 94610","Near+la+Salle+Ave+and+King+Ave+Piedmont,+CA+94610",37.815256,-122.223950
1,https://www.pawboost.com/landing/pet/-kF0OoECq...,5406229,Unknown,Male,FOUND,Very friendly and sweet.,"Near 47th Ave & Ygnacio Ave Oakland, CA 94601","Near+47th+Ave+and+Ygnacio+Ave+Oakland,+CA+94601",37.774218,-122.207961
2,https://www.pawboost.com/landing/pet/FOLymmKQb...,5445880,Unknown,Unknown,FOUND,Multi colored cat,"Near 67th Ave & Avenal Ave Oakland, CA 94605","Near+67th+Ave+and+Avenal+Ave+Oakland,+CA+94605",37.765243,-122.186799
3,https://www.pawboost.com/landing/pet/T1W061MY0...,5412303,Unknown,Female,FOUND,Orange white has collar with pink bow. Super f...,"Near Parker Ave & Outlook Ave Oakland, CA 94605","Near+Parker+Ave+and+Outlook+Ave+Oakland,+CA+94605",37.768372,-122.165566
4,https://www.pawboost.com/landing/pet/qXEhVVZT3...,5412800,Unknown,Female,FOUND,Super friendly and obviously lost from a famil...,"Near Parker Ave & Outlook Ave Oakland, CA 94605","Near+Parker+Ave+and+Outlook+Ave+Oakland,+CA+94605",37.768372,-122.165566
5,https://www.pawboost.com/landing/pet/M-8bw1Bz6...,5370159,Unknown,Female,FOUND,Calico Cat that I have been feeding every nigh...,"Near Coach Dr & Phaeton Dr Oakland, CA 94605","Near+Coach+Dr+and+Phaeton+Dr+Oakland,+CA+94605",37.773161,-122.139061
6,https://www.pawboost.com/landing/pet/QQXhIwWuq...,5447641,Unknown,Unknown,FOUND,Black cat with white chest and white paws.,"Potrero Hill San Francisco, CA 94107","Potrero+Hill+San+Francisco,+CA+94107",37.761824,-122.398587
7,https://www.pawboost.com/landing/pet/ZsDLbMKFt...,5470380,Unknown,Unknown,FOUND,NEIGHBORHOOD LAST SEEN,"Harlan-Estabrook-Castro San Leandro, CA 94577","Harlan-Estabrook-Castro+San+Leandro,+CA+94577",37.717148,-122.155611
8,https://www.pawboost.com/landing/pet/nXeOeZaF9...,5509632,Unknown,Male,FOUND,Sweet male cat who loves to purr and cuddle ju...,"Alamo Square San Francisco, CA 94117","Alamo+Square+San+Francisco,+CA+94117",37.777499,-122.433252
9,https://www.pawboost.com/landing/pet/mLVcy1OwJ...,5516397,Unknown,Male,FOUND,Orange tabby (assume it's a male). Very old/si...,"Alamo Square San Francisco, CA 94115","Alamo+Square+San+Francisco,+CA+94115",37.777499,-122.433252


In [10]:
details_reunited

,url,ref_no,pet_name,gender,status,description,address,address_clean,latitude,longitude
0,https://www.pawboost.com/landing/pet/oq00DJiba...,5485310,Jax,Male,SAFE,"Jax is a tabby colored cat with a white belly,...","Oakland, CA 94610","Oakland,+CA+94610",37.810448,-122.239864
1,https://www.pawboost.com/landing/pet/8voHTy_iZ...,5450995,Chad,Male,SAFE,Black and White cat (sometimes called a tuxedo...,"Oakland, CA 94606","Oakland,+CA+94606",37.794409,-122.245536
2,https://www.pawboost.com/landing/pet/uEmdVyeiT...,5429986,Justice,Male,SAFE,NEIGHBORHOOD LAST SEEN,"Oakland, CA 94607","Oakland,+CA+94607",37.813468,-122.307917
3,https://www.pawboost.com/landing/pet/MHr71Jv79...,5507282,Loki,Male,SAFE,Loki has one eye but is fully blind. His tail ...,"Oakland, CA 94610","Oakland,+CA+94610",37.810448,-122.239864
4,https://www.pawboost.com/landing/pet/mi8ie5x3y...,5396921,Luna,Female,SAFE,Microchipped. Spayed. Wearing a shiny purple c...,"Oakland, CA 94611","Oakland,+CA+94611",37.833628,-122.202983
5,https://www.pawboost.com/landing/pet/aFvQ2ZwZ-...,5467952,Romelu,Male,SAFE,"8 month old brown tabby kitten, no collar, but...","Oakland, CA 94608","Oakland,+CA+94608",37.837959,-122.282402
6,https://www.pawboost.com/landing/pet/bWsWhpu63...,5454579,Liba,Female,SAFE,Tortiseshell short hair female with light eyeb...,"Oakland, CA 94602","Oakland,+CA+94602",37.801552,-122.214332
7,https://www.pawboost.com/landing/pet/Uo6Jdtp_I...,5392063,Calico Bunny,Female,SAFE,"Calico kitty, 4 years old, weighs about 5lbs. ...","Oakland, CA 94601","Oakland,+CA+94601",37.772927,-122.214332
8,https://www.pawboost.com/landing/pet/JkKj-VEdK...,5450424,Raja,Female,SAFE,She’s a little sweet calico kitty with a cute ...,"Alameda, CA 94501","Alameda,+CA+94501",37.771217,-122.282402
9,https://www.pawboost.com/landing/pet/2HDWoUbGw...,5380186,Emi,Female,SAFE,10 year old blue cream point Siamese mix. Very...,"Berkeley, CA 94705","Berkeley,+CA+94705",37.864615,-122.234190


In [11]:
details_lost.to_csv("details_lost.csv", index=True, header=True)
details_found.to_csv("details_found.csv", index=True, header=True)
details_reunited.to_csv("details_reunited.csv", index=True, header=True)